## include libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

## download train set

In [ ]:
train_set=torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])     
)

## visilaize data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
loader=torch.utils.data.DataLoader(train_set, batch_size=10)

image,label=next(iter(train_set))
plt.imshow(image.squeeze(),cmap='gray')



In [ ]:
batch=next(iter(loader))
images,labels= batch
grid=torchvision.utils.make_grid(images, nrow=5)
plt.imshow(np.transpose(grid))

## neural network 

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.con1=nn.Conv2d(in_channels=1, out_channels=6 ,kernel_size=5)
        self.con2=nn.Conv2d(in_channels=6, out_channels=12  ,kernel_size=5)
        
        self.fc1=nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2=nn.Linear(in_features=120 , out_features=60)
        self.out=nn.Linear(in_features=60, out_features=10)
    
    def forward(self,t):
        #1st layer
        t=t
        #2nd layer
        t=self.con1(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2, stride=2)
        #3rd layer
        
        t=self.con2(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2, stride=2)
        
        #4th layer
        t=t.reshape(-1,12*4*4)
        t=self.fc1(t)
        t=F.relu(t)
        
        #5th layer
       
        t=self.fc2(t)
        t=F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        return t

In [ ]:
network=Network()


## prediction

In [ ]:
torch.set_grad_enabled(False)
pred=network(image)
pred.argmax(dim=1).eq(label)

In [ ]:
preds=network(images)
preds.argmax(dim=1)

## training, backpropagation and loss calculation

In [ ]:
loader=torch.utils.data.DataLoader(train_set,batch_size=100, shuffle=True)
torch.set_grad_enabled(True)
optimizer=optim.Adam(network.parameters(), lr=0.01)

for ep in range(3):
    total_loss=0
    for batch in loader:
        images,labels=batch
        preds=network(images)
        loss=F.cross_entropy(preds,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
    print("epoch", ep, "loss:", total_loss)
        
        
        

## confusion matrix

In [ ]:
def get_all_preds(model, laoder):
    all_pred=torch.tensor([])
    for batch in laoder:
        images,labels=batch
        preds=network(images)
        all_pred=torch.cat((all_pred,preds),dim=0)
    return all_pred

In [ ]:
with torch.no_grad():
    prediction_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
    train_preds = get_all_preds(network, prediction_loader)

In [ ]:
train_preds.shape

In [ ]:
cmt=torch.zeros([10,10])
cmt

In [ ]:
stacked=torch.stack((train_set.targets,train_preds.argmax(dim=1)),dim=1)
for p in stacked:
    j,k=p.tolist()
    cmt[j,k]=cmt[j,k]+1

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cmt
#plot_confusion_matrix(cmt, train_set.classes)

## tensor borad for visializtion of result

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
tb=SummaryWriter()
network1=Network()
train_loader = torch.utils.data.DataLoader(
    train_set
    ,batch_size=100
    ,shuffle=True
)
images, labels=next(iter(train_loader))
grid=torchvision.utils.make_grid(images)
tb.add_image('images',grid)
tb.add_graph(network1,images)
tb.close()

## Access the TensorBoard UI by browsing to http://localhost:6006 

In [ ]:
loader=torch.utils.data.DataLoader(train_set,batch_size=100, shuffle=True)
torch.set_grad_enabled(True)
optimizer=optim.Adam(network.parameters(), lr=0.01)
tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network1, images)

for ep in range(3):
    total_loss=0
    total_correct=0
    for batch in loader:
        images,labels=batch
        preds=network(images)
        loss=F.cross_entropy(preds,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss+=loss.item()
        total_correct += preds.argmax(dim=1).eq(labels).sum().item()
    tb.add_scalar('Loss', total_loss, ep)
    tb.add_scalar('Number Correct', total_correct, ep)
    tb.add_histogram('conv1.bias', network.con1.bias, ep)
    print("epoch", ep, "loss:", total_loss)
tb.close()

## Experimenting With Hyperparameter Values

In [ ]:
batch_size_list = [100, 1000, 10000]
lr_list = [.01, .001, .0001, .00001]
for batch_size in batch_size_list:
    for lr in lr_list:
        network = Network()

        train_loader = torch.utils.data.DataLoader(
            train_set, batch_size=batch_size
        )
        optimizer = optim.Adam(
            network.parameters(), lr=lr
        )

        images, labels = next(iter(train_loader))
        grid = torchvision.utils.make_grid(images)

        comment=f' batch_size={batch_size} lr={lr}'
        tb = SummaryWriter(comment=comment)
        tb.add_image('images', grid)
        tb.add_graph(network, images)

        for epoch in range(1):
            total_loss = 0
            total_correct = 0
            for batch in train_loader:
                images, labels = batch # Get Batch
                preds = network(images) # Pass Batch
                loss = F.cross_entropy(preds, labels) # Calculate Loss
                optimizer.zero_grad() # Zero Gradients
                loss.backward() # Calculate Gradients
                optimizer.step() # Update Weights

                total_loss += loss.item() * batch_size
                total_correct += preds.argmax(dim=1).eq(labels).sum().item()

            tb.add_scalar(
                'Loss', total_loss, epoch
            )
            tb.add_scalar(
                'Number Correct', total_correct, epoch
            )
            tb.add_scalar(
                'Accuracy', total_correct / len(train_set), epoch
            )

            for name, param in network.named_parameters():
                tb.add_histogram(name, param, epoch)
                tb.add_histogram(f'{name}.grad', param.grad, epoch)

            print(
                "epoch", epoch
                ,"total_correct:", total_correct
                ,"loss:", total_loss
            )  
        tb.close()
